In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './Preprocessing')
import numpy as np
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from imgDataGenerator import *
from preprocessing import *
import tensorflow as tf
import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import model_from_json

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [ ]:
#data dimension will be: #bag * #frame * 244 * 244 * 3
#simple implementation of CNN>LSTM without multires


In [3]:
#Note for Test, Training, and Validation folders, they all need to have same categories
parentPath='/home/livelab/Desktop/NewResizedBinned'
checkAndGeneratePath(parentPath)
testPath='/home/livelab/Desktop/NewResizedBinned/Test'
trainPath='/home/livelab/Desktop/NewResizedBinned/Training'
validPath='/home/livelab/Desktop/NewResizedBinned/Validation'

trainingGen=imgDataGenerator(trainPath)
validGen=imgDataGenerator(validPath)

Length Check on Folder Succeed


In [ ]:
testPath='/home/billy/Desktop/VideoClasBinned/Test'
trainPath='/home/billy/Desktop/VideoClasBinned/Training'
validPath='/home/billy/Desktop/VideoClasBinned/Validation'
combo=prepdata(trainPath)
len(combo)
a,b=zip(*combo)
b=np.argmax(a,axis=1)
b=b.tolist()
for i in range(24):
    print(b.count(i),i)

In [ ]:
#Using TimeDistributed Wrapper
inputData=Input(shape=(None, 256, 256, 3))
model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 256, 256, 3))
#for layer in model.layers[:278]: #until mixed9_0, untrainable
#   layer.trainable = False
layer_name = 'mixed9'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
# for layer in intermediate_layer_model.layers: #until mixed9_0, untrainable
#   layer.trainable = False
encoded_frame = TimeDistributed(intermediate_layer_model)(inputData)

encoded_frame = TimeDistributed(Flatten())(encoded_frame)
# encoded_frame = TimeDistributed(Dense(64))(encoded_frame)
# encoded_frame = BatchNormalization()(encoded_frame)
# encoded_frame = TimeDistributed(LeakyReLU(alpha=0.1))(encoded_frame)
intermediateOut = CuDNNLSTM(128, return_sequences = True)(encoded_frame)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = CuDNNLSTM(64, return_sequences = True)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)

intermediateOut = TimeDistributed(Dense(32))(intermediateOut)
intermediateOut = TimeDistributed(BatchNormalization())(intermediateOut)
intermediateOut = TimeDistributed(LeakyReLU(alpha=0.1))(intermediateOut)
intermediateOut = TimeDistributed(Dropout(0.4))(intermediateOut)

# intermediateOut = Dense(32)(intermediateOut)
# intermediateOut = BatchNormalization()(intermediateOut)
# intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)

intermediateOut = TimeDistributed(Dropout(0.4))(intermediateOut)
prediction = TimeDistributed(Dense(24, activation='softmax'))(intermediateOut)
optmr = keras.optimizers.adam(lr=0.005, decay = 5e-4)
model_final = Model(input = [inputData], output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])
filepath = "saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='auto', period=1)
callbacks_list = [checkpoint]

In [5]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 64, 64, 48)  12224     
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 196608)      0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, None, 128)         100729856 
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 128)         512       
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, None, 64)          49664     
_________________________________________________________________
batch_normalization_3 (Batch (None, None, 64)          256       
__________

In [8]:
with tf.device('GPU:0'):
    model_final.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    callbacks=callbacks_list,
                    use_multiprocessing=True,
                    epochs = 40,
                    workers=6)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
1129/1129 [==============================] - 394s 349ms/step - loss: 3.1812 - acc: 0.0599 - val_loss: 3.1493 - val_acc: 0.0859

Epoch 00001: saving model to saved-model-01-0.09.hdf5
Epoch 2/40
1128/1129 [============================>.] - ETA: 0s - loss: 3.0947 - acc: 0.0641
Epoch 00001: saving model to saved-model-01-0.09.hdf5
1129/1129 [==============================] - 369s 327ms/step - loss: 3.0943 - acc: 0.0645 - val_loss: 4.1560 - val_acc: 0.0052

Epoch 00002: saving model to saved-model-02-0.01.hdf5
Epoch 3/40
1129/1129 [==============================] - 382s 338ms/step - loss: 3.0285 - acc: 0.0748 - val_loss: 3.1603 - val_acc: 0.0953

Epoch 00003: saving model to saved-model-03-0.10.hdf5
Epoch 4/40
 696/1129 [=================>............] - ETA: 2:19 - loss: 2.9955 - acc: 0.0904

Process ForkPoolWorker-41:
Process ForkPoolWorker-45:
Process ForkPoolWorker-42:
Process ForkPoolWorker-40:
Process ForkPoolWorker-43:
Process ForkPoolWorker-47:
Process ForkPoolWorker-48:
Process ForkPoolWorker-46:
Process ForkPoolWorker-44:
Process ForkPoolWorker-37:
Process ForkPoolWorker-38:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/ho

KeyboardInterrupt
KeyboardInterrupt
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process ForkPoolWorker-39:
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/queues.py", line 358, in put
    obj = _ForkingPickler.dumps(obj)
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
KeyboardInterrupt


KeyboardInterrupt: 

In [7]:
#Using TimeDistributed Wrapper and freeze all CNN layer to mix8
inputData=Input(shape=( 256, 256, 3))
inputDataTime=Input(shape=(None, 256, 256, 3))
tower_1 = Conv2D(8, (1,1), padding='same')(inputData)
tower_1 = LeakyReLU(alpha=0.1)(tower_1)
tower_1 = Conv2D(8, (3,3), padding='same')(tower_1)
tower_1 = LeakyReLU(alpha=0.1)(tower_1)
tower_1 = MaxPooling2D()(tower_1)
tower_2 = Conv2D(8, (1,1), padding='same')(inputData)
tower_2 = LeakyReLU(alpha=0.1)(tower_2)
tower_2 = Conv2D(8, (5,5), padding='same')(tower_2)
tower_2 = LeakyReLU(alpha=0.1)(tower_2)
tower_2 = MaxPooling2D()(tower_2)
tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(inputData)
tower_3 = Conv2D(8, (1,1), padding='same')(tower_3)
tower_3 = LeakyReLU(alpha=0.1)(tower_3)
tower_3 = MaxPooling2D()(tower_3)
mix0 = concatenate([tower_1, tower_2, tower_3], axis = 3)
tower_4 = Conv2D(16, (1,1), padding='same')(mix0)
tower_4 = LeakyReLU(alpha=0.1)(tower_4)
tower_4 = Conv2D(16, (3,3), padding='same')(tower_4)
tower_4 = LeakyReLU(alpha=0.1)(tower_4)
tower_4 = MaxPooling2D()(tower_4)
tower_5 = Conv2D(16, (1,1), padding='same')(mix0)
tower_5 = LeakyReLU(alpha=0.1)(tower_5)
tower_5 = Conv2D(16, (5,5), padding='same')(tower_5)
tower_5 = LeakyReLU(alpha=0.1)(tower_5)
tower_5 = MaxPooling2D()(tower_5)
tower_6 = MaxPooling2D((3,3), strides=(1,1), padding='same')(mix0)
tower_6 = Conv2D(16, (1,1), padding='same')(tower_6)
tower_6 = LeakyReLU(alpha=0.1)(tower_6)
tower_6 = MaxPooling2D()(tower_6)                 
mix1 = concatenate([tower_4, tower_5, tower_6], axis = 3)
intermediate_layer_model = Model(inputs=inputData,
                                 outputs=mix1)
encoded_frame = TimeDistributed(intermediate_layer_model)(inputDataTime)
encoded_frame=TimeDistributed(Flatten())(encoded_frame)

intermediateOut = TimeDistributed(Dense(64))(encoded_frame)
intermediateOut = TimeDistributed(BatchNormalization())(intermediateOut)
intermediateOut = TimeDistributed(LeakyReLU(alpha=0.1))(intermediateOut)
intermediateOut = TimeDistributed(Dropout(0.4))(intermediateOut)

intermediateOut = CuDNNLSTM(128, return_sequences = True)(encoded_frame)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = CuDNNLSTM(64, return_sequences = True)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)

# intermediateOut = Dense(64)(intermediateOut)
# intermediateOut = BatchNormalization()(intermediateOut)
# intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
# intermediateOut = Dropout(0.4)(intermediateOut)

intermediateOut = TimeDistributed(Dense(32))(intermediateOut)
intermediateOut = TimeDistributed(BatchNormalization())(intermediateOut)
intermediateOut = TimeDistributed(LeakyReLU(alpha=0.1))(intermediateOut)

intermediateOut = TimeDistributed(Dropout(0.4))(intermediateOut)
prediction = TimeDistributed(Dense(24, activation='softmax'))(intermediateOut)
optmr_freeze = keras.optimizers.adam(lr=0.005, decay = 5e-4)
model_final = Model(input = [inputDataTime], output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr_freeze, metrics=["accuracy"])
filepath = "saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='auto', period=1)
callbacks_list = [checkpoint]

In [ ]:
with tf.device('GPU:0'):
    model_final_freeze.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    use_multiprocessing=True,
                    epochs = 20,
                    workers=2)

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [ ]:
model_json = model_final.to_json()
with open("modelmix3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_final.save_weights("modelmix3.h5")

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_final = model_from_json(loaded_model_json)
# load weights into new model
model_final.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
#manually test model using testset
testGen=imgDataGenerator(testPath)
correct=0
laxcorrect=0
with tf.device('GPU:0'):
    for i in range(124):
        x,y=testGen.__getitem__(i)
        pred=model_final.predict(x)
        pred=pred.reshape((len(pred[0]),24))
        spred=np.sum(pred,axis=0)
        aspred=np.argsort(-spred)
#         print(aspred[:3])
#         print(aspred)
#         print(np.argmax(spred))
#         print(spred)
#         print(np.argmax(y[0][0]))
#         print('**')
        if(np.argmax(y[0][0]) in aspred[:2]):
            correct+=1
        if(np.argmax(y[0][0]) in aspred[:4]):
            laxcorrect+=1    
print('Accuracy of Testing Set: ',correct/124, laxcorrect/124)